# Bitcoin price forecasting with PySpark - Data crawling
## Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



## Global constants, dependencies, libraries and tools

In [23]:
GDRIVE_DIR = "/content/drive"
GDRIVE_DATASET_RAW_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/raw"
GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_30min"
GDRIVE_DATASET_NAME_EXT = "/" + GDRIVE_DATASET_NAME + ".parquet"
GDRIVE_DATASET = GDRIVE_DATASET_RAW_DIR + GDRIVE_DATASET_NAME_EXT

In [1]:
import pandas as pd
import functools

from google.colab import drive

from datetime import date

In [24]:
# Point Colaboratory to our Google Drive
from google.colab import drive

# Define GDrive paths
drive.mount(GDRIVE_DIR)

Mounted at /content/drive


##  Define metrics and parameterss

In this section we are going to define the parameters used to collect the data and the metrics used. I will consider Bitcoin data for 10 years, starting from 2012-01-01 through 2022-12-31.

Note that since the Blockchain.com API allows retreiving data with a maximum timespan equal to 6 years, I manually computed the continue date so that I could make a second API call to get the remaining data.

Regarding the metrics, I chose the ones that seemed most relevant to me, containing both price statistics but also technical features of Bitcoin's blockchain.

In [25]:
# Get the current date
current_date = date.today()

# Print the current date
print(current_date)

2023-08-20


In [26]:
# Define the parameters
timespan = "6years" # Duration of the data (because the Max timespan == 6years)
start_date = "2016-01-01"
continue_date = "2021-12-31" # The continue date (manually calculate the continue_date)
end_date = str(current_date)

# Metrics considered
metrics = [
          ##Currency Statistics##
            "market-price", # Market Price: The average USD market price across major bitcoin exchanges.
            "trade-volume", #Exchange Trade Volume (USD): The total USD value of trading volume on major bitcoin exchanges.

          ##Block Details##
            "blocks-size", # Blockchain Size (MB): The total size of the blockchain minus database indexes in megabytes.
            "avg-block-size", # Average Block Size (MB): The average block size over the past 24 hours in megabytes.
            "n-transactions-total", # Total Number of Transactions: The total number of transactions on the blockchain.
            "n-transactions-per-block", # Average Transactions Per Block: The average number of transactions per block over the past 24 hours.

          ##Mining Information##
            "hash-rate", # Total Hash Rate (TH/s): The estimated number of terahashes per second the bitcoin network is performing in the last 24 hours.
            "difficulty", # Network Difficulty (T): A relative measure of how difficult it is to mine a new block for the blockchain.
            "miners-revenue", # Miners Revenue (USD): Total value of coinbase block rewards and transaction fees paid to miners.
            "transaction-fees-usd", # Total Transaction Fees (USD): The total USD value of all transaction fees paid to miners. This does not include coinbase block rewards.

          ##Network Activity##
            "n-unique-addresses", # The total number of unique addresses used on the blockchain.
            "n-transactions", # Confirmed Transactions Per Day: The total number of confirmed transactions per day.
            "estimated-transaction-volume-usd" # Estimated Transaction Value (USD): The total estimated value in USD of transactions on the blockchain. This does not include coins returned as change.
]

## Retreiving data

In this section we are going to make the call to the Blockchain.com API to retrieve the data.

In [27]:
def data_crawler(timespan, metrics, start_date, continue_date, end_date):
    # API Info
    url1 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={start_date}&format=csv'
    url2 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={continue_date}&format=csv'

    # Obtain Data
    data1 = pd.read_csv(url1,names=['timestamp',metrics])
    data2 = pd.read_csv(url2,names=['timestamp',metrics])

    # Concat by rows
    all_data = pd.concat([data1,data2])

    # Transform "timestamp" to datetime type
    all_data['timestamp'] = pd.to_datetime(all_data["timestamp"])

    # Keep the same end date with Bitcoin data
    all_data = all_data[(all_data['timestamp'] < end_date)]

    return all_data

In [28]:
# Merge the data
merge = functools.partial(pd.merge, on='timestamp')

# Gain Blockchain Data from Blockchain.com API
df1 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df1

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07
2,2016-01-03,432.76,1.287046e+06,54754.876205,0.554656,101427625,983.503448,7.694240e+05,1.038803e+11,1.720316e+06,11009.108568,335666.0,142608.0,6.764693e+07
3,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07
4,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2776,2023-08-15,29413.80,8.428877e+07,503564.249680,1.655429,879116599,3409.725146,4.453489e+08,5.239118e+13,3.191514e+07,536032.415741,786113.0,583063.0,2.891347e+09
2777,2023-08-16,29169.09,7.327130e+07,503847.295317,1.685333,879700157,3477.779141,4.245139e+08,5.239118e+13,3.058697e+07,530443.731065,669749.0,566878.0,2.674221e+09
2778,2023-08-17,28701.48,1.030395e+08,504122.471424,1.644294,880269214,3219.705128,4.062832e+08,5.239118e+13,2.842468e+07,538982.205443,769086.0,502274.0,4.488093e+09
2779,2023-08-18,26662.05,3.168022e+08,504378.306602,1.630966,880768245,3044.654676,3.620088e+08,5.239118e+13,2.343302e+07,578088.172336,783855.0,423207.0,3.939446e+09


In [29]:
# Check duplicated rows
len(df1['timestamp'].unique())

2781

Due to a problem with the Blockchain.com API, I was forced to make an additional call to retrieve capitalization and total circulating data that will be added to the currency statistics to get a single dataset.

In [30]:
# Retrieving market capitalization and total circulating data
metrics = [
  "total-bitcoins", # Total Circulating Bitcoin: The total number of mined bitcoin that are currently circulating on the network.
  "market-cap", # Market Capitalization (USD): The total USD value of bitcoin in circulation.
  ]

merge = functools.partial(pd.merge, on='timestamp')
df2 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df2

,timestamp,total-bitcoins,market-cap
0,2016-01-01 00:03:21,15029575.00,6.474140e+09
1,2016-01-02 15:54:15,15035125.00,6.499685e+09
2,2016-01-04 02:26:37,15040650.00,6.458253e+09
3,2016-01-05 09:50:06,15046150.00,6.489359e+09
4,2016-01-06 21:03:44,15051750.00,6.472554e+09
...,...,...,...
3005,2023-08-18 05:44:35,19460768.75,5.146400e+11
3006,2023-08-18 17:15:52,19461131.25,5.012798e+11
3007,2023-08-19 02:35:08,19461493.75,5.073806e+11
3008,2023-08-19 14:02:31,19461856.25,5.049379e+11


In [31]:
# Check duplicated rows
len(df2['timestamp'].unique())

3010

In [32]:
# Wipe off the timestamp's h:m:s.
df2['timestamp'] = pd.to_datetime(df2["timestamp"]).dt.normalize()

# Drop the duplicates in column "timestamp", keep the last value
df2.drop_duplicates(subset="timestamp", keep="last", inplace=True)

In [33]:
df2

,timestamp,total-bitcoins,market-cap
0,2016-01-01,15029575.00,6.474140e+09
1,2016-01-02,15035125.00,6.499685e+09
2,2016-01-04,15040650.00,6.458253e+09
3,2016-01-05,15046150.00,6.489359e+09
4,2016-01-06,15051750.00,6.472554e+09
...,...,...,...
2998,2023-08-15,19458218.75,5.718381e+11
3001,2023-08-16,19459312.50,5.670054e+11
3004,2023-08-17,19460406.25,5.427507e+11
3006,2023-08-18,19461131.25,5.012798e+11


In [34]:
# Check duplicated rows
len(df2['timestamp'].unique())

2101

In [35]:
# Add the market capitalization and total circulating data
all_data = pd.merge(df1, df2, how="inner", on='timestamp')
all_data = all_data.interpolate(method='ffill')
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07,15029575.00,6.474140e+09
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07,15035125.00,6.499685e+09
2,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07,15040650.00,6.458253e+09
3,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08,15046150.00,6.489359e+09
4,2016-01-06,432.43,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08,15051750.00,6.472554e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,2023-08-15,29413.80,8.428877e+07,503564.249680,1.655429,879116599,3409.725146,4.453489e+08,5.239118e+13,3.191514e+07,536032.415741,786113.0,583063.0,2.891347e+09,19458218.75,5.718381e+11
2093,2023-08-16,29169.09,7.327130e+07,503847.295317,1.685333,879700157,3477.779141,4.245139e+08,5.239118e+13,3.058697e+07,530443.731065,669749.0,566878.0,2.674221e+09,19459312.50,5.670054e+11
2094,2023-08-17,28701.48,1.030395e+08,504122.471424,1.644294,880269214,3219.705128,4.062832e+08,5.239118e+13,2.842468e+07,538982.205443,769086.0,502274.0,4.488093e+09,19460406.25,5.427507e+11
2095,2023-08-18,26662.05,3.168022e+08,504378.306602,1.630966,880768245,3044.654676,3.620088e+08,5.239118e+13,2.343302e+07,578088.172336,783855.0,423207.0,3.939446e+09,19461131.25,5.012798e+11


In [36]:
# Check nan value
all_data[all_data.isnull().T.any()]

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap


In [37]:
# Check duplicated rows
len(all_data['timestamp'].unique())

2097

In [38]:
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07,15029575.00,6.474140e+09
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07,15035125.00,6.499685e+09
2,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07,15040650.00,6.458253e+09
3,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08,15046150.00,6.489359e+09
4,2016-01-06,432.43,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08,15051750.00,6.472554e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,2023-08-15,29413.80,8.428877e+07,503564.249680,1.655429,879116599,3409.725146,4.453489e+08,5.239118e+13,3.191514e+07,536032.415741,786113.0,583063.0,2.891347e+09,19458218.75,5.718381e+11
2093,2023-08-16,29169.09,7.327130e+07,503847.295317,1.685333,879700157,3477.779141,4.245139e+08,5.239118e+13,3.058697e+07,530443.731065,669749.0,566878.0,2.674221e+09,19459312.50,5.670054e+11
2094,2023-08-17,28701.48,1.030395e+08,504122.471424,1.644294,880269214,3219.705128,4.062832e+08,5.239118e+13,2.842468e+07,538982.205443,769086.0,502274.0,4.488093e+09,19460406.25,5.427507e+11
2095,2023-08-18,26662.05,3.168022e+08,504378.306602,1.630966,880768245,3044.654676,3.620088e+08,5.239118e+13,2.343302e+07,578088.172336,783855.0,423207.0,3.939446e+09,19461131.25,5.012798e+11


In [39]:
new_columns = ['timestamp', 'market-price', 'total-bitcoins', 'market-cap'] + [col for col in all_data.columns if col not in ['timestamp', 'market-price', 'total-bitcoins', 'market-cap']]
all_data = all_data.reindex(columns=new_columns)
all_data

,timestamp,market-price,total-bitcoins,market-cap,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2016-01-01,430.89,15029575.00,6.474140e+09,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07
1,2016-01-02,434.75,15035125.00,6.499685e+09,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07
2,2016-01-04,430.78,15040650.00,6.458253e+09,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07
3,2016-01-05,434.17,15046150.00,6.489359e+09,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08
4,2016-01-06,432.43,15051750.00,6.472554e+09,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,2023-08-15,29413.80,19458218.75,5.718381e+11,8.428877e+07,503564.249680,1.655429,879116599,3409.725146,4.453489e+08,5.239118e+13,3.191514e+07,536032.415741,786113.0,583063.0,2.891347e+09
2093,2023-08-16,29169.09,19459312.50,5.670054e+11,7.327130e+07,503847.295317,1.685333,879700157,3477.779141,4.245139e+08,5.239118e+13,3.058697e+07,530443.731065,669749.0,566878.0,2.674221e+09
2094,2023-08-17,28701.48,19460406.25,5.427507e+11,1.030395e+08,504122.471424,1.644294,880269214,3219.705128,4.062832e+08,5.239118e+13,2.842468e+07,538982.205443,769086.0,502274.0,4.488093e+09
2095,2023-08-18,26662.05,19461131.25,5.012798e+11,3.168022e+08,504378.306602,1.630966,880768245,3044.654676,3.620088e+08,5.239118e+13,2.343302e+07,578088.172336,783855.0,423207.0,3.939446e+09


Once we have the daily dataset we will go to sample it at a frequency of 1 minute (1T) using the resample method. This means that the data will be organized in 1-minute time-frame, and an interpolation method will be used to fill in any missing data or holes in the DataFrame by estimating missing values based on the surrounding known values.

In [40]:
# Upsampling to 1min by interpolate
all_data.set_index('timestamp', inplace=True)
all_data_30m = all_data.resample('30T').interpolate()
all_data_30m

,market-price,total-bitcoins,market-cap,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
timestamp,,,,,,,,,,,,,,,
2016-01-01 00:00:00,430.890000,1.502958e+07,6.474140e+09,2.860153e+06,54604.791735,0.493407,1.011557e+08,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.000000,124092.000000,5.834626e+07
2016-01-01 00:30:00,430.970417,1.502969e+07,6.474672e+09,2.834859e+06,54606.164986,0.495206,1.011583e+08,921.463506,6.973443e+05,1.038803e+11,1.557199e+06,8887.551433,264578.937500,124611.708333,5.818972e+07
2016-01-01 01:00:00,431.050833,1.502981e+07,6.475204e+09,2.809565e+06,54607.538237,0.497004,1.011608e+08,923.727011,6.975595e+05,1.038803e+11,1.559629e+06,8992.039015,266036.875000,125131.416667,5.803318e+07
2016-01-01 01:30:00,431.131250,1.502992e+07,6.475736e+09,2.784271e+06,54608.911488,0.498803,1.011634e+08,925.990517,6.977747e+05,1.038803e+11,1.562059e+06,9096.526597,267494.812500,125651.125000,5.787664e+07
2016-01-01 02:00:00,431.211667,1.503004e+07,6.476268e+09,2.758977e+06,54610.284739,0.500602,1.011659e+08,928.254023,6.979898e+05,1.038803e+11,1.564490e+06,9201.014178,268952.750000,126170.833333,5.772010e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-18 22:00:00,26098.300000,1.946213e+07,5.073891e+11,1.799027e+08,504586.224709,1.693243,8.811569e+08,3657.042652,3.643961e+08,5.239118e+13,2.323950e+07,446745.598113,641302.333333,511732.250000,1.418873e+09
2023-08-18 22:30:00,26085.487500,1.946215e+07,5.075279e+11,1.767913e+08,504590.950120,1.694658,8.811657e+08,3670.960560,3.644504e+08,5.239118e+13,2.323511e+07,443760.539608,638062.500000,513744.187500,1.361587e+09
2023-08-18 23:00:00,26072.675000,1.946217e+07,5.076668e+11,1.736800e+08,504595.675532,1.696074,8.811745e+08,3684.878469,3.645046e+08,5.239118e+13,2.323071e+07,440775.481103,634822.666667,515756.125000,1.304301e+09


## Output

In this last section we are going to save the dataset we just created to the Google Drive.

In [41]:
# Output the 1h data
all_data_30m.to_parquet(GDRIVE_DATASET)